<a href="https://colab.research.google.com/github/Nithyarajoman/Complexity-calculation-functions/blob/main/Models_evaluated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [ ]:
# MAC CALCULATION
# Function to calculate return MAC from each layer
def mac_each_layer(l):
    o_shape, i_shape, strides, ks, filters = ['', '', ''], ['', '', ''], [1, 1], [0, 0], [0, 0]        
    macc = 0
    
    if ('Dense' in str(l)):
        i_shape = l.input.shape[1:4].as_list()[0]
        o_shape = l.output.shape[1:4].as_list()
        macc = (o_shape[0] * i_shape)

    
    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((filters * ks[0] * ks[1] * i_shape[2]) * (
                (i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((ks[0] * ks[1] * i_shape[2]) * ((i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    return macc

# Function to pass layers to the calculating function and to get the macc of each layer
def MAC_summary(model):
  print('%25s | %6s' % ('Layer Name', 'MACs'))
  print('*' * 100)
  t_macc = 0

  for l in model.layers:
    if (type(l) in [models.Sequential, keras.Model]):
      for ib_layer in l.layers:
        mac_obtained = mac_each_layer(ib_layer)
        print('%25s(%5s) | %5.4f' % (
                l.name,ib_layer.name, mac_obtained ))
        t_macc += mac_obtained  

    else:
      mac_obtained = mac_each_layer(l)
      print('%25s | %5.4f' % (
                l.name, mac_obtained ))
      
    t_macc += mac_obtained
  print('\nTotal MACs operations in the complete model: %10.8f\n' % (t_macc))
  return

In [ ]:
# EFFECTIVE DATA COMMUNICATION

# Function to print the effective data communicated 
def data_summary(model):

  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"
    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)
    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)
  
  # Compiling the cloned model and take the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  aux = m2.evaluate(x_test[0:1000], y_test[0:1000]) 
  
  print('%40s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  print("*" * 150)
  for l in m2.metrics:
    print('%40s | %5.4f' % (l.name, l.result().numpy()))
  return


In [ ]:
# ImageNet classification dataset for Resnet18 and ResNet50
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data() #Size is (, 32,32,3)

In [ ]:
# Calculations for Resnet18 CNN model
!pip install image-classifiers
from tensorflow.keras.applications import imagenet_utils
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model_ResNet18 = ResNet18(include_top=False,
                        input_tensor=None,
                        input_shape=(
                            32, 32, 3),
                        pooling=None)
MAC_summary(model_ResNet18)
data_summary(model_ResNet18)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
               Layer Name |   MACs
****************************************************************************************************
                     data | 0.0000
                  bn_data | 0.0000
       zero_padding2d_162 | 0.0000
                    conv0 | 3396288.0000
                      bn0 | 0.0000
                    relu0 | 0.0000
       zero_padding2d_163 | 0.0000
                 pooling0 | 0.0000
         stage1_unit1_bn1 | 0.0000
       stage1_unit1_relu1 | 0.0000
       zero_padding2d_164 | 0.0000
       stage1_unit1_conv1 | 3686400.0000
         stage1_unit1_bn2 | 0.0000
       stage1_unit1_relu2 | 0.0000
       zero_padding2d_165 | 0.0000
       stage1_unit1_conv2 | 3686400.0000
          stage1_unit1_sc | 262144.0000
                   add_72 | 0.0000
         stage1_unit2_bn1 | 0.0000
       stage1_unit2_relu1 | 0.0000
       zero_padding2d_166 | 0.0000
       

In [ ]:
# Calculations for Resnet50 CNN model
from tensorflow.keras.applications.resnet50 import ResNet50
model_Resnet50 = ResNet50(include_top=False,
                        weights='imagenet',
                        input_shape=(32, 32, 3),
                        pooling=None)
MAC_summary(model_Resnet50)
data_summary(model_Resnet50)

94765736/94765736 [==============================] - 1s 0us/step
               Layer Name |   MACs
****************************************************************************************************
                  input_1 | 0.0000
                conv1_pad | 0.0000
               conv1_conv | 3396288.0000
                 conv1_bn | 0.0000
               conv1_relu | 0.0000
                pool1_pad | 0.0000
               pool1_pool | 0.0000
      conv2_block1_1_conv | 262144.0000
        conv2_block1_1_bn | 0.0000
      conv2_block1_1_relu | 0.0000
      conv2_block1_2_conv | 2359296.0000
        conv2_block1_2_bn | 0.0000
      conv2_block1_2_relu | 0.0000
      conv2_block1_0_conv | 1048576.0000
      conv2_block1_3_conv | 1048576.0000
        conv2_block1_0_bn | 0.0000
        conv2_block1_3_bn | 0.0000
         conv2_block1_add | 0.0000
         conv2_block1_out | 0.0000
      conv2_block2_1_conv | 1048576.0000
        conv2_block2_1_bn | 0.0000
      conv2_block2_1_relu | 0